# 데이터 전처리

데이터 출처 : [서울 열린 데이터 광장](https://github.com/vuski/SeoulBikeStationLocation)   
대여소 데이터 출처 : https://github.com/vuski/SeoulBikeStationLocation

In [2]:
# 패키지
import os

import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime

In [3]:
# 데이터 불러오기
path = '../data/'
df_2020 = pd.read_csv(path+'data_2020.csv')

C:\Users\82103\anaconda3\envs\quantum\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [188]:
point = pd.read_csv(path + '대여소위치.tsv', encoding = 'cp949', delimiter = '\t')

In [5]:
print(f'df_2020: {df_2020.shape}')
print(f'point: {point.shape}')

df_2020: (17421645, 12)
point: (2290, 6)


In [6]:
df_2020.head()

,Unnamed: 0,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,0,SPB-04061,2020-01-01 00:01:04,429,송도병원,2,2020-01-01 00:04:21,00372,약수역 3번출구 뒤,8,2.0,0.0
1,1,SPB-06686,2020-01-01 00:02:53,1637,KT 노원점 건물 앞,14,2020-01-01 00:04:48,01656,중앙하이츠 아파트 입구,9,1.0,350.0
2,2,SPB-15937,2020-01-01 00:01:21,1924,삼부르네상스파크빌,10,2020-01-01 00:05:51,01955,디지털입구 교차로,7,4.0,800.0
3,3,SPB-14805,2020-01-01 00:03:12,437,대흥역 1번출구,1,2020-01-01 00:05:54,00126,서강대 후문 옆,18,2.0,0.0
4,4,SPB-09038,2020-01-01 00:01:11,1168,마곡엠밸리10단지 앞,5,2020-01-01 00:05:57,01152,마곡역교차로,2,4.0,660.0


In [7]:
# 불필요한 열 삭제
df_2020 = df_2020.drop(columns='Unnamed: 0', axis=1)

In [8]:
columns_name = ['bike_id', 'rental_date', 'rental_point', 'rental_point_name', 'rental_point_rack', 'return_date', 'return_point', 'return_point_name', 'return_point_rack', 'hour_use', 'distance' ]
df_2020.columns = columns_name

In [9]:
data = df_2020.copy()

## 데이터 전처리
### 1. 대여소,반납소명 오류로 한칸씩 밀린 경우  
* distance가 null 값인 경우

In [155]:
error_data = pd.DataFrame(data[data.distance.isnull() == True])

In [156]:
#  rental_point_name에서 오류 난 경우
rental_error = error_data[error_data.rental_point_name.str.contains('\?')]
return_error = error_data[error_data.return_point_name.str.contains('\?')]

In [157]:
rental_error.head(3)

,bike_id,rental_date,rental_point,rental_point_name,rental_point_rack,return_date,return_point,return_point_name,return_point_rack,hour_use,distance
47102,SPB-09846,2020-01-03 06:51:33,1162,"공항초등학교?풍覺?,8""",2020-01-03 06:56:34,02701,마곡나루역 5번출구 뒤편,8,4,830.0,NaN
80712,SPB-12017,2020-01-04 11:38:07,3509,"세종사이?測淪閨?,11""",2020-01-04 11:41:35,03518,군자역 7번출구뒤,7,3,760.0,NaN
203728,SPB-07700,2020-01-10 17:24:40,1984,"구?慣맡?,7""",2020-01-10 17:56:46,02135,신림역 5번출구,2,30,5360.0,NaN


In [158]:
error_data.head(3)

,bike_id,rental_date,rental_point,rental_point_name,rental_point_rack,return_date,return_point,return_point_name,return_point_rack,hour_use,distance
47102,SPB-09846,2020-01-03 06:51:33,1162,"공항초등학교?풍覺?,8""",2020-01-03 06:56:34,02701,마곡나루역 5번출구 뒤편,8,4,830.0,NaN
62480,SPB-02297,2020-01-03 17:37:10,1428,원묵고등학교,6,2020-01-03 18:00:43,01428,"원묵?玆紵閨?,6""",23,0.0,NaN
63437,SPB-16990,2020-01-03 18:11:04,1986,태평양물산빌딩,5,2020-01-03 18:21:37,01916,"?막适峠閨낼?,2""",10,1570.0,NaN


In [159]:
print(f'총 에러 데이터 개수: {error_data.shape[0]}')
print(f'대여에서 밀린 개수: {rental_error.shape[0]}')
print(f'반납에서 밀린 개수: {return_error.shape[0]}')
print(f'에러데이터와 대여/반납 데이터 차이 : {error_data.shape[0]-rental_error.shape[0]-return_error.shape[0]}')

총 에러 데이터 개수: 111
대여에서 밀린 개수: 47
반납에서 밀린 개수: 64
에러데이터와 대여/반납 데이터 차이 : 0


In [250]:
for idx ,row in rental_error.iterrows():
    if row.rental_point_name.find('/?'):
        row[5:] =  row[4:-1] # 한 칸 밀기
        row.rental_point_rack = pd.to_numeric(row.rental_point_name[-2]) # 거치대 수 : 대여소명 마지막 숫자
        error_data.loc[idx] = row # 인덱스로 raw_data에서 변경
        # df_2020.loc[idx] = row

In [161]:
for idx ,row in rental_error.iterrows():
    if row.rental_point_name.find('/?'):
        row[5:] =  row[4:-1] # 한 칸 밀기
        row.rental_point_rack = pd.to_numeric(row.rental_point_name[-2]) # 거치대 수 : 대여소명 마지막 숫자
        error_data.loc[idx] = row # 인덱스로 raw_data에서 변경

In [162]:
for idx ,row in return_error.iterrows():
    if row.return_point_name.find('/?'):
        row[9:] =  row[8:-1] # 한 칸 밀기
        row.return_point_rack = int(row.return_point_name[-2]) # 거치대 수 : 대여소명 마지막 숫자
        error_data.loc[idx] = row # 인덱스로 raw_data에서 변경

### 2.  대여소 id
-> error 데이터 대여소 명 변경

In [189]:
# point.reset_index(drop=True)
point  = point.set_index(['stationIndex'],drop=True)

In [165]:
# 타입 변경
error_data.rental_point = error_data.rental_point.apply(int)
error_data.return_point = error_data.return_point.apply(int)

In [191]:
point.loc[1986, 'name']

'1986. 태평양물산빌딩'

In [232]:
for i in point.loc[point.name.str.contains('\.'),'name'].index:
   point.loc[i,'name'] = point.loc[i,'name'].split('.')[-1]

In [242]:
for idx, row in e.iterrows():
    if row.rental_point_name.find('\?'):
        row.rental_point_name = point.loc[row.rental_point, 'name']
        error_data.loc[idx, 'rental_point_name'] = row.rental_point_name

In [244]:
for idx, row in e.iterrows():
    if row.return_point_name.find('\?'):
        row.return_point_name = point.loc[row.return_point, 'name']
        error_data.loc[idx, 'return_point_name'] = row.return_point_name

In [259]:
for i in tqdm(error_data.index, desc="progress:"):
    # print(i)
    data.loc[i] = error_data.loc[i]


progress:: 100%|██████████| 111/111 [06:29<00:00,  3.51s/it]


In [260]:
data.distance.isnull().sum()

0

In [261]:
data

,bike_id,rental_date,rental_point,rental_point_name,rental_point_rack,return_date,return_point,return_point_name,return_point_rack,hour_use,distance
0,SPB-04061,2020-01-01 00:01:04,429,송도병원,2,2020-01-01 00:04:21,00372,약수역 3번출구 뒤,8,2.0,0.0
1,SPB-06686,2020-01-01 00:02:53,1637,KT 노원점 건물 앞,14,2020-01-01 00:04:48,01656,중앙하이츠 아파트 입구,9,1.0,350.0
2,SPB-15937,2020-01-01 00:01:21,1924,삼부르네상스파크빌,10,2020-01-01 00:05:51,01955,디지털입구 교차로,7,4.0,800.0
3,SPB-14805,2020-01-01 00:03:12,437,대흥역 1번출구,1,2020-01-01 00:05:54,00126,서강대 후문 옆,18,2.0,0.0
4,SPB-09038,2020-01-01 00:01:11,1168,마곡엠밸리10단지 앞,5,2020-01-01 00:05:57,01152,마곡역교차로,2,4.0,660.0
...,...,...,...,...,...,...,...,...,...,...,...
17421640,SPB-32394,2020-05-14 20:12,502,뚝섬유원지역 1번출구 앞,0,2020-05-14 20:14,502,뚝섬유원지역 1번출구 앞,0,2.0,0.0
17421641,SPB-31606,2020-05-14 19:25,128,신촌역(2호선) 1번출구 옆,0,2020-05-14 20:14,2013,장승배기역 5번출구,0,50.0,0.0
17421642,SPB-30988,2020-05-14 19:25,278,쌍용플레티넘오피스텔,0,2020-05-14 20:14,227,양평2나들목 보행통로 입구,0,50.0,0.0
17421643,SPB-31046,2020-05-14 19:27,913,이마트 은평점,0,2020-05-14 20:14,123,문화촌 공원,0,48.0,0.0
